In [1]:
import pandas as pd 
import pyodbc
import sqlalchemy
import datetime as dt 
import numpy as np

#Se definen las conexiones

def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'jrivas',
                        'XXXXX',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res
    
def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res

In [2]:
query_1 = """ 
    

select distinct YEAR(VisitStartDateTime) as year, MONTH(VisitStartDateTime) as month, DAY(VisitStartDateTime) as day, sessionuid, sceneuid, VisitStartDateTime, locationId, SceneType, SubSceneType, TipoMueble, FabMueble, productid, Puertas_Coca, Puertas_Competencia, Puertas_Cadena from dbo.as_ir_report    
where YEAR(VisitStartDateTime) = '2021' and SceneType = 'Enfriador' and TipoMueble = 'Enfriadores Piso de Ventas' 


        """

In [3]:
df = pd.DataFrame(get_query(query_1))

In [4]:
df['date_format']=pd.to_datetime(df[['year','month','day']])

In [5]:
df=df.rename(columns={"sessionuid": "SessionUid"})

In [6]:
df.duplicated(subset='SessionUid', keep='first').sum()

1227342

In [7]:
df=df.drop_duplicates(subset=['SessionUid'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13580 entries, 0 to 1181728
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   year                 13580 non-null  int64         
 1   month                13580 non-null  int64         
 2   day                  13580 non-null  int64         
 3   SessionUid           13580 non-null  object        
 4   sceneuid             13580 non-null  object        
 5   VisitStartDateTime   13580 non-null  datetime64[ns]
 6   locationId           13580 non-null  int64         
 7   SceneType            13580 non-null  object        
 8   SubSceneType         13580 non-null  object        
 9   TipoMueble           13580 non-null  object        
 10  FabMueble            13580 non-null  object        
 11  productid            13580 non-null  int64         
 12  Puertas_Coca         13580 non-null  int64         
 13  Puertas_Competencia  13580 no

In [8]:
query_2 = """ 
   
select distinct SessionUid, LocationId, UserId, DateId, ChainDsc, Format, Clasification, ICE, coolers_fl_ko, enfriador_Competencia_FL, enfriador_Cadena_FL, is_reported from dbo.r_autoservicios_v3    


        """

In [9]:
df2 = pd.DataFrame(get_query(query_2))

In [10]:
df2.head()

,SessionUid,LocationId,UserId,DateId,ChainDsc,Format,Clasification,ICE,coolers_fl_ko,enfriador_Competencia_FL,enfriador_Cadena_FL,is_reported
0,0001df49-6f2d-4795-83fd-e961ffac4f3a,7277764,45793,20210305,SUPER AKI,MINI,BÁSICA,105.1600,1,0,0,1
1,0002f44a-be74-4b2b-954c-71ddaedde55f,6855087,45842,20210121,MERCADO,BODEGA,BÁSICA,68.8100,3,2,0,1
2,0003e1de-616b-4ccb-8af2-1f8de9957b66,6585042,45877,20210206,CASA LEY,SUPER,BÁSICA,101.6500,2,1,0,1
3,00043727-9410-49ff-8624-e571b3ad86db,6585252,45894,20201216,HIPERMERCADO,HIPER,BÁSICA,94.2500,6,2,0,1
4,0004b846-2f8d-4cbb-b927-8ba9767e4334,6860791,45929,20210825,BODEGA AURRERA,BODEGA,DESARROLLO,106.2315,1,1,0,1


In [11]:
dff = pd.merge(df2, df, on='SessionUid', how='inner')
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   SessionUid                13580 non-null  object        
 1   LocationId                13580 non-null  int64         
 2   UserId                    13580 non-null  int64         
 3   DateId                    13580 non-null  object        
 4   ChainDsc                  13571 non-null  object        
 5   Format                    13580 non-null  object        
 6   Clasification             13571 non-null  object        
 7   ICE                       13580 non-null  float64       
 8   coolers_fl_ko             13580 non-null  int64         
 9   enfriador_Competencia_FL  13580 non-null  int64         
 10  enfriador_Cadena_FL       13580 non-null  int64         
 11  is_reported               13580 non-null  int64         
 12  year              

In [12]:
table = pd.pivot_table(dff, values=['Puertas_Coca','Puertas_Competencia', 'Puertas_Cadena'], index=['UserId', 'LocationId', 'is_reported', 'date_format', 'month', 'day', 'SessionUid'],
                     fill_value=0)
table=pd.DataFrame(table)
table

Puertas_Cadena  \
UserId LocationId is_reported date_format month day SessionUid                                             
43976  6473384    0           2021-06-01  6     1   d2df32c6-f357-4009-ae85-5d6c38809ea7               0   
43978  6473384    0           2021-06-01  6     1   1fffc695-3619-49c5-b01b-d4510c2ec5fd               0   
                              2021-06-17  6     17  f5caea7c-a7f2-4c14-982e-8094c25eb31b               0   
43979  6473384    0           2021-06-04  6     4   d391984a-ee97-4df1-b9c0-481f941a2295               0   
                              2021-06-23  6     23  70448b47-3f90-41a7-9f43-53b98b1aecbe               0   
...                                                                                                  ...   
58238  6853974    1           2021-10-06  10    6   c9684d57-f1d6-4a79-95dd-4353f575d5e4               8   
       6853977    1           2021-10-06  10    6   2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a               0   
       6861884    1           2021-10-02  10    2   b10a9d79-30cf-4cee-b465-5107706d8409               0   
       7277818    1           2021-10-12  10    12  3a603e1c-fce7-40ac-8244-268593dca0d0               0   
       9813407    1           2021-09-28  9     28  37aa6a75-1fcf-4c71-963e-cc64213b8f1c               0   

                                                                                          Puertas_Coca  \
UserId LocationId is_reported date_format month day SessionUid                                           
43976  6473384    0           2021-06-01  6     1   d2df32c6-f357-4009-ae85-5d6c38809ea7             7   
43978  6473384    0           2021-06-01  6     1   1fffc695-3619-49c5-b01b-d4510c2ec5fd            29   
                              2021-06-17  6     17  f5caea7c-a7f2-4c14-982e-8094c25eb31b            33   
43979  6473384    0           2021-06-04  6     4   d391984a-ee97-4df1-b9c0-481f941a2295            26   
                              2021-06-23  6     23  70448b47-3f90-41a7-9f43-53b98b1aecbe            34   
...                                                                                                ...   
58238  6853974    1           2021-10-06  10    6   c9684d57-f1d6-4a79-95dd-4353f575d5e4            24   
       6853977    1           2021-10-06  10    6   2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a            27   
       6861884    1           2021-10-02  10    2   b10a9d79-30cf-4cee-b465-5107706d8409            36   
       7277818    1           2021-10-12  10    12  3a603e1c-fce7-40ac-8244-268593dca0d0            23   
       9813407    1           2021-09-28  9     28  37aa6a75-1fcf-4c71-963e-cc64213b8f1c            30   

                                                                                          Puertas_Competencia  
UserId LocationId is_reported date_format month day SessionUid                                                 
43976  6473384    0           2021-06-01  6     1   d2df32c6-f357-4009-ae85-5d6c38809ea7                    2  
43978  6473384    0           2021-06-01  6     1   1fffc695-3619-49c5-b01b-d4510c2ec5fd                   11  
                              2021-06-17  6     17  f5caea7c-a7f2-4c14-982e-8094c25eb31b                    4  
43979  6473384    0           2021-06-04  6     4   d391984a-ee97-4df1-b9c0-481f941a2295                   12  
                              2021-06-23  6     23  70448b47-3f90-41a7-9f43-53b98b1aecbe                    0  
...                                                                                                       ...  
58238  6853974    1           2021-10-06  10    6   c9684d57-f1d6-4a79-95dd-4353f575d5e4                   18  
       6853977    1           2021-10-06  10    6   2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a                   26  
       6861884    1           2021-10-02  10    2   b10a9d79-30cf-4cee-b465-5107706d8409                   52  
       7277818    1           2021-10-12  10    12  3a603e1c-fce7-40ac-824

In [13]:
table = table.reset_index()
table

,UserId,LocationId,is_reported,date_format,month,day,SessionUid,Puertas_Cadena,Puertas_Coca,Puertas_Competencia
0,43976,6473384,0,2021-06-01,6,1,d2df32c6-f357-4009-ae85-5d6c38809ea7,0,7,2
1,43978,6473384,0,2021-06-01,6,1,1fffc695-3619-49c5-b01b-d4510c2ec5fd,0,29,11
2,43978,6473384,0,2021-06-17,6,17,f5caea7c-a7f2-4c14-982e-8094c25eb31b,0,33,4
3,43979,6473384,0,2021-06-04,6,4,d391984a-ee97-4df1-b9c0-481f941a2295,0,26,12
4,43979,6473384,0,2021-06-23,6,23,70448b47-3f90-41a7-9f43-53b98b1aecbe,0,34,0
...,...,...,...,...,...,...,...,...,...,...
13575,58238,6853974,1,2021-10-06,10,6,c9684d57-f1d6-4a79-95dd-4353f575d5e4,8,24,18
13576,58238,6853977,1,2021-10-06,10,6,2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a,0,27,26
13577,58238,6861884,1,2021-10-02,10,2,b10a9d79-30cf-4cee-b465-5107706d8409,0,36,52
13578,58238,7277818,1,2021-10-12,10,12,3a603e1c-fce7-40ac-8244-268593dca0d0,0,23,44


In [14]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   UserId               13580 non-null  int64         
 1   LocationId           13580 non-null  int64         
 2   is_reported          13580 non-null  int64         
 3   date_format          13580 non-null  datetime64[ns]
 4   month                13580 non-null  int64         
 5   day                  13580 non-null  int64         
 6   SessionUid           13580 non-null  object        
 7   Puertas_Cadena       13580 non-null  int64         
 8   Puertas_Coca         13580 non-null  int64         
 9   Puertas_Competencia  13580 non-null  int64         
dtypes: datetime64[ns](1), int64(8), object(1)
memory usage: 1.0+ MB


In [15]:
moni=table.groupby(['UserId','LocationId', 'SessionUid', 'is_reported', 'date_format', 'month', 'day'])['Puertas_Coca', 'Puertas_Competencia', 'Puertas_Cadena'].sum()

moni  

<ipython-input-15-3618e5c41397>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  moni=table.groupby(['UserId','LocationId', 'SessionUid', 'is_reported', 'date_format', 'month', 'day'])['Puertas_Coca', 'Puertas_Competencia', 'Puertas_Cadena'].sum()


Puertas_Coca  \
UserId LocationId SessionUid                           is_reported date_format month day                 
43976  6473384    d2df32c6-f357-4009-ae85-5d6c38809ea7 0           2021-06-01  6     1               7   
43978  6473384    1fffc695-3619-49c5-b01b-d4510c2ec5fd 0           2021-06-01  6     1              29   
                  f5caea7c-a7f2-4c14-982e-8094c25eb31b 0           2021-06-17  6     17             33   
43979  6473384    70448b47-3f90-41a7-9f43-53b98b1aecbe 0           2021-06-23  6     23             34   
                  c0da7d72-8f06-4e55-9dbd-37ab66c0f58b 0           2021-07-09  7     9              32   
...                                                                                                ...   
58238  6853974    c9684d57-f1d6-4a79-95dd-4353f575d5e4 1           2021-10-06  10    6              24   
       6853977    2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a 1           2021-10-06  10    6              27   
       6861884    b10a9d79-30cf-4cee-b465-5107706d8409 1           2021-10-02  10    2              36   
       7277818    3a603e1c-fce7-40ac-8244-268593dca0d0 1           2021-10-12  10    12             23   
       9813407    37aa6a75-1fcf-4c71-963e-cc64213b8f1c 1           2021-09-28  9     28             30   

                                                                                          Puertas_Competencia  \
UserId LocationId SessionUid                           is_reported date_format month day                        
43976  6473384    d2df32c6-f357-4009-ae85-5d6c38809ea7 0           2021-06-01  6     1                      2   
43978  6473384    1fffc695-3619-49c5-b01b-d4510c2ec5fd 0           2021-06-01  6     1                     11   
                  f5caea7c-a7f2-4c14-982e-8094c25eb31b 0           2021-06-17  6     17                     4   
43979  6473384    70448b47-3f90-41a7-9f43-53b98b1aecbe 0           2021-06-23  6     23                     0   
                  c0da7d72-8f06-4e55-9dbd-37ab66c0f58b 0           2021-07-09  7     9                      1   
...                                                                                                       ...   
58238  6853974    c9684d57-f1d6-4a79-95dd-4353f575d5e4 1           2021-10-06  10    6                     18   
       6853977    2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a 1           2021-10-06  10    6                     26   
       6861884    b10a9d79-30cf-4cee-b465-5107706d8409 1           2021-10-02  10    2                     52   
       7277818    3a603e1c-fce7-40ac-8244-268593dca0d0 1           2021-10-12  10    12                    44   
       9813407    37aa6a75-1fcf-4c71-963e-cc64213b8f1c 1           2021-09-28  9     28                    29   

                                                                                          Puertas_Cadena  
UserId LocationId SessionUid                           is_reported date_format month day                  
43976  6473384    d2df32c6-f357-4009-ae85-5d6c38809ea7 0           2021-06-01  6     1                 0  
43978  6473384    1fffc695-3619-49c5-b01b-d4510c2ec5fd 0           2021-06-01  6     1                 0  
                  f5caea7c-a7f2-4c14-982e-8094c25eb31b 0           2021-06-17  6     17                0  
43979  6473384    70448b47-3f90-41a7-9f43-53b98b1aecbe 0           2021-06-23  6     23                0  
                  c0da7d72-8f06-4e55-9dbd-37ab66c0f58b 0           2021-07-09  7     9                 0  
...                                                                                                  ...  
58238  6853974    c9684d57-f1d6-4a79-95dd-4353f575d5e4 1           2021-10-06  10    6                 8  
       6853977    2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a 1           2021-10-06  10    6                 0  
       6861884    b10a9d79-30cf-4cee-b465-5107706d8409 1           2021-10-02  10    2                 0  
       7277818    3a603e1c-fce7-40ac-8244-268593dca0d0 1        

In [16]:
table2 = moni.reset_index()
table2

,UserId,LocationId,SessionUid,is_reported,date_format,month,day,Puertas_Coca,Puertas_Competencia,Puertas_Cadena
0,43976,6473384,d2df32c6-f357-4009-ae85-5d6c38809ea7,0,2021-06-01,6,1,7,2,0
1,43978,6473384,1fffc695-3619-49c5-b01b-d4510c2ec5fd,0,2021-06-01,6,1,29,11,0
2,43978,6473384,f5caea7c-a7f2-4c14-982e-8094c25eb31b,0,2021-06-17,6,17,33,4,0
3,43979,6473384,70448b47-3f90-41a7-9f43-53b98b1aecbe,0,2021-06-23,6,23,34,0,0
4,43979,6473384,c0da7d72-8f06-4e55-9dbd-37ab66c0f58b,0,2021-07-09,7,9,32,1,0
...,...,...,...,...,...,...,...,...,...,...
13575,58238,6853974,c9684d57-f1d6-4a79-95dd-4353f575d5e4,1,2021-10-06,10,6,24,18,8
13576,58238,6853977,2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a,1,2021-10-06,10,6,27,26,0
13577,58238,6861884,b10a9d79-30cf-4cee-b465-5107706d8409,1,2021-10-02,10,2,36,52,0
13578,58238,7277818,3a603e1c-fce7-40ac-8244-268593dca0d0,1,2021-10-12,10,12,23,44,0


In [17]:
table2=table2.sort_values(['LocationId', 'date_format'])
table2

,UserId,LocationId,SessionUid,is_reported,date_format,month,day,Puertas_Coca,Puertas_Competencia,Puertas_Cadena
11178,52049,6188351,6fbb054a-4bb4-4660-b055-9be64eb62409,1,2021-05-21,5,21,31,62,0
3720,45847,6188351,7dbcf5ac-dfb7-4a2c-9c74-62ee7957352e,1,2021-06-03,6,3,30,55,1
3719,45847,6188351,549e6699-f573-4a84-ba02-53f02693fe43,1,2021-07-06,7,6,27,66,0
11179,52049,6188351,a38e7b33-6b0f-42e3-a8bc-7991cb5fb7e0,1,2021-08-14,8,14,43,79,0
11176,52049,6188351,2c000823-ce4b-4d38-a733-37a9e06b29c7,1,2021-09-09,9,9,45,115,0
...,...,...,...,...,...,...,...,...,...,...
97,45788,10051235,ba1f20e9-b741-45d1-8ece-b145d4ce2a49,1,2021-07-10,7,10,22,28,0
1056,45793,10051235,4e2bf448-b164-4323-969c-5897eeee33ca,1,2021-08-07,8,7,34,16,2
13497,57355,10051235,5da0103c-71bc-4cba-b706-d5af482cb097,1,2021-09-04,9,4,24,18,0
13498,57355,10051235,8095be30-6805-4f57-b5cb-d55e56edc677,1,2021-10-02,10,2,27,17,0


In [18]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13580 entries, 11178 to 576
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   UserId               13580 non-null  int64         
 1   LocationId           13580 non-null  int64         
 2   SessionUid           13580 non-null  object        
 3   is_reported          13580 non-null  int64         
 4   date_format          13580 non-null  datetime64[ns]
 5   month                13580 non-null  int64         
 6   day                  13580 non-null  int64         
 7   Puertas_Coca         13580 non-null  int64         
 8   Puertas_Competencia  13580 non-null  int64         
 9   Puertas_Cadena       13580 non-null  int64         
dtypes: datetime64[ns](1), int64(8), object(1)
memory usage: 1.1+ MB


In [19]:
table2["Coca-Cola"] = table2["Puertas_Coca"].shift(1)
table2["Cadena"] = table2["Puertas_Competencia"].shift(1)
table2["Competencia"] = table2["Puertas_Cadena"].shift(1)
table2.dtypes

UserId                          int64
LocationId                      int64
SessionUid                     object
is_reported                     int64
date_format            datetime64[ns]
month                           int64
day                             int64
Puertas_Coca                    int64
Puertas_Competencia             int64
Puertas_Cadena                  int64
Coca-Cola                     float64
Cadena                        float64
Competencia                   float64
dtype: object

In [20]:
table2 = table2.fillna(0)

In [21]:
table2 = table2.astype({"Coca-Cola": int, "Competencia": int, "Cadena": int, "LocationId": object})

In [22]:
table2["LocationIdChanged"] = table2["LocationId"].shift(1, fill_value=table2["LocationId"].head(1)) != table2["LocationId"]
table2.dtypes

UserId                          int64
LocationId                     object
SessionUid                     object
is_reported                     int64
date_format            datetime64[ns]
month                           int64
day                             int64
Puertas_Coca                    int64
Puertas_Competencia             int64
Puertas_Cadena                  int64
Coca-Cola                       int32
Cadena                          int32
Competencia                     int32
LocationIdChanged                bool
dtype: object

In [23]:
table2["LocationIdChanged"] = table2["LocationIdChanged"].astype(object)
table2.dtypes

UserId                          int64
LocationId                     object
SessionUid                     object
is_reported                     int64
date_format            datetime64[ns]
month                           int64
day                             int64
Puertas_Coca                    int64
Puertas_Competencia             int64
Puertas_Cadena                  int64
Coca-Cola                       int32
Cadena                          int32
Competencia                     int32
LocationIdChanged              object
dtype: object

In [24]:
for OutletCodeChanged in table2:
      s = table2['LocationIdChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_CC = 'change'
              
          else: 
              Flag_CC = table2["Puertas_Coca"] - table2["Coca-Cola"]

              table2['Flag_CC']=Flag_CC

In [25]:
for OutletCodeChanged in table2:
      s = table2['LocationIdChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_Comp = 'change'
              
          else: 
              Flag_Comp = table2["Puertas_Competencia"] - table2["Competencia"]

              table2['Flag_Comp']=Flag_Comp

In [26]:
for OutletCodeChanged in table2:
      s = table2['LocationIdChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_Prop = 'change'
              
          else: 
              Flag_Prop = table2["Puertas_Cadena"] - table2["Cadena"]

              table2['Flag_Cad']=Flag_Prop

In [27]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13580 entries, 11178 to 576
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   UserId               13580 non-null  int64         
 1   LocationId           13580 non-null  object        
 2   SessionUid           13580 non-null  object        
 3   is_reported          13580 non-null  int64         
 4   date_format          13580 non-null  datetime64[ns]
 5   month                13580 non-null  int64         
 6   day                  13580 non-null  int64         
 7   Puertas_Coca         13580 non-null  int64         
 8   Puertas_Competencia  13580 non-null  int64         
 9   Puertas_Cadena       13580 non-null  int64         
 10  Coca-Cola            13580 non-null  int32         
 11  Cadena               13580 non-null  int32         
 12  Competencia          13580 non-null  int32         
 13  LocationIdChanged    13580 no

In [28]:
table_final = table2[['UserId','LocationId','SessionUid', 'is_reported', 'date_format', 'month', 'day', 'LocationIdChanged', 'Puertas_Coca', 'Flag_CC', 'Puertas_Competencia', 'Flag_Comp', 'Puertas_Cadena', 'Flag_Cad']]
table_final

,UserId,LocationId,SessionUid,is_reported,date_format,month,day,LocationIdChanged,Puertas_Coca,Flag_CC,Puertas_Competencia,Flag_Comp,Puertas_Cadena,Flag_Cad
11178,52049,6188351,6fbb054a-4bb4-4660-b055-9be64eb62409,1,2021-05-21,5,21,False,31,31,62,62,0,0
3720,45847,6188351,7dbcf5ac-dfb7-4a2c-9c74-62ee7957352e,1,2021-06-03,6,3,False,30,-1,55,55,1,-61
3719,45847,6188351,549e6699-f573-4a84-ba02-53f02693fe43,1,2021-07-06,7,6,False,27,-3,66,65,0,-55
11179,52049,6188351,a38e7b33-6b0f-42e3-a8bc-7991cb5fb7e0,1,2021-08-14,8,14,False,43,16,79,79,0,-66
11176,52049,6188351,2c000823-ce4b-4d38-a733-37a9e06b29c7,1,2021-09-09,9,9,False,45,2,115,115,0,-79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,45788,10051235,ba1f20e9-b741-45d1-8ece-b145d4ce2a49,1,2021-07-10,7,10,True,22,15,28,28,0,-16
1056,45793,10051235,4e2bf448-b164-4323-969c-5897eeee33ca,1,2021-08-07,8,7,False,34,12,16,16,2,-26
13497,57355,10051235,5da0103c-71bc-4cba-b706-d5af482cb097,1,2021-09-04,9,4,False,24,-10,18,16,0,-16
13498,57355,10051235,8095be30-6805-4f57-b5cb-d55e56edc677,1,2021-10-02,10,2,False,27,3,17,17,0,-18


In [29]:
table_final = table_final.fillna(0)

In [ ]:
print("flags-as - Borrando los datos de la tabla")
query_clear_PV_as = """ 
    
DELETE FROM reportes.Puertas_PV_tot_as

        """

In [ ]:
def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'jrivas',
                        'XXXXX',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res
    
def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res

eng = get_engine()
conn = eng.connect()
conn.execute(query_clear_PV_as)
conn.close()
#get_query(query_clear_Exh_as_cc_comp)


print("flags-as - Llenando la tabla")

In [30]:
engine = get_engine()
table_final.to_sql('Puertas_PV_tot_as', con=engine, schema='reportes', if_exists='append', index=False)